In [1]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import pandas as pd
import os, os.path
import re
import networkx as nx
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore') # avoid warnings showing up

In [2]:
def read_data(path): #input the path to the directory with data
    frames = []
    _,_, files = next(os.walk(path)) #create a list of all datafile names
    for file in tqdm(files): #for every file in directory
        with open(path+"/"+file) as f: #read each file
            dataframe = pd.read_json(f) #convert file to dataframe
     
        frames.append(dataframe) #append each dataframe to list
    data = pd.concat(frames, sort=False) #make it one big dataframe
    
    return data
    

In [3]:
data = read_data("scraped")

In [4]:
subset = data.iloc[:50]

In [24]:
article_sentences = subset.iloc[2]["Body"].split(". ")
article_sentences

[' Køb abonnement Den familieorienterede type: Slægt skal følge slægters gang For denne type er det naturligt at lægge et billede af barnet ud hurtigt efter fødslen',
 'Nogle offentliggør scanningsfoto og viser med tekster som »vores lille nye familiemedlem«, at de allerede inden fødslen tænker på sig selv som en familie',
 'Der postes løbende billeder af barnet',
 'Af hverdagsøjeblikke, der signalerer intimitet, samvær og opbygning af relationer',
 'Opslag er ofte knyttet til noget rituelt/ceremonielt og milepæle: Hjemme fra fødegangen',
 'Første tur i barnevognen',
 'En måned gammel',
 'Debut til babysvømning',
 'Barnedåb',
 'Halvårsfødselsdag',
 'Fejringer',
 'Højtider',
 'Formålet er at etablere en familiefortælling på Facebook og skabe en familiær identitet ved at dele og vise sit nye familieliv frem',
 'Man markerer et tilhørsforhold til en bestemt slægt',
 'Man går op i traditioner og har et hverdagsliv, der er centreret omkring familien',
 'Der er både kvinder og mænd i denne g

In [29]:
def read_sentences(text):
    text_sentences = text.split(". ")
    sentences = []
    for sentence in text_sentences: 
#         print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()
        
    return sentences

In [30]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords == None: 
        stopwords = []
        
    sent1 = [word.lower() for word in sent1]
    sent2 = [word.lower() for word in sent2]
    
    all_words = list(set(sent1 + sent2))
    
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    
    #create vector for the first sentence
    for word in sent1: 
        if word in stopwords: 
            continue
        vector1[all_words.index(word)] += 1
    
    #create vector for the second sentence
    for word in sent2: 
        if word in stopwords: 
            continue
        vector2[all_words.index(word)] += 1
        
    return 1 - cosine_distance(vector1, vector2)

In [31]:
def similarity_matrix(sentences, stop_words): 
    
    #Empty similarity matrix
    sim_matrix = np.zeros((len(sentences), len(sentences)))
    
    for index1 in range(len(sentences)):
        for index2 in range(len(sentences)):
            if index1 == index2: #ignore if they're the same sentence
                continue
            sim_matrix[index1][index2] = sentence_similarity(sentences[index1], sentences[index2], stop_words)
    
    return sim_matrix

In [32]:
# sents = read_sentences(subset.iloc[0]["Body"])
# similarity_matrix(sents, stopwords.words("danish"))

array([[0. , 0. , 0. , 0. , 0. , 0.3, 0. , 0. ],
       [0. , 0. , 0. , 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.3, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])

In [37]:
def create_summary(text, top_n=5): 
    
    stop_words = stopwords.words("danish")
    summarize_text = []
    
    #Split text into sentences
    sentences = read_sentences(text)
    
    #Create similarity matrix across sentences
    sentence_sim_matrix = similarity_matrix(sentences, stop_words)
    
    #rank sentences
    
    sentence_sim_graph = nx.from_numpy_array(sentence_sim_matrix)
    scores = nx.pagerank(sentence_sim_graph)
    
    #sort scores and pick top sentences
    ranked_sentence = sorted(((scores[i], s) for i,s in enumerate(sentences)), reverse=True)
#     print("Indices of top ranked sentence order are: ", ranked_sentence)
    
    for i in range(top_n): 
        summarize_text.append(" ".join(ranked_sentence[i][1]))
    
    print("Original text: \n\n", text, "\n")
    
    print("Text summary: \n\n", ". ".join(summarize_text))
    
    return None

In [38]:
create_summary(subset.iloc[0]["Body"])

Original text: 

  FOR ABONNENTER »Der var overhovedet ingen beskeder over højttaleren« »Det har været ganske forfærdeligt. Er blevet meget forsinket på arbejde og hjem. Overfyldte toge, aflyste toge, små toge (ikke lange), toge der pludselig ikke holdt på visse stationer, så man blev over 30 min. forsinket. Det ville klæde DSB at give penge retur, når man har periodekort og i den grad ikke får den vare, man har betalt for. Ofte er der overhovedet ingen beskeder over højttaleren, og på skiltningen på perronen går afgangstiden ned fra 10 min. til 0 og derpå forfra uden tog og uden forklaring. Prøv selv en tur kl. 7.30 en hverdagsmorgen!!!!!« 

Text summary: 

 forsinket. Ofte er der overhovedet ingen beskeder over højttaleren, og på skiltningen på perronen går afgangstiden ned fra 10 min. Er blevet meget forsinket på arbejde og hjem.  FOR ABONNENTER »Der var overhovedet ingen beskeder over højttaleren« »Det har været ganske forfærdeligt. til 0 og derpå forfra uden tog og uden forklaring